In [ ]:
# 설명 : 동적페이지에 키워드가 있는 지 확인 -> 앞에서 찾은 부분이 정적 페이지에 있는 지 확인 - > 있으면 추출

In [17]:
# keyword 예시 : ~남음, ~개, ~보고 있~, ~까지, ~명이 담음, ~보는 

In [34]:
#standard library
import time, os, random

#pandas import
import pandas as pd
import time
import os

#Selenium import
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException  # 예외 처리 추가
from selenium.common.exceptions import StaleElementReferenceException #예외 처리 추가
from selenium.webdriver.chrome.options import Options # user-agent 변경

#BeautifulSoup import
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

# Function

In [2]:
# keyword가 포함된 block 불러와서 csv 파일에 저장하는 함수
def search_keyword(keywords, url, driver):
    
    #1. url을 연다
    print("Dynamic URL open : "+ url)
    driver.get(url) #동적 페이지

    time.sleep(random.randrange(10,20)) #로딩되는데 걸리는 시간(10초 이상)
    
    # XPath OR 조건 생성
    xpath_conditions = " or ".join([f"contains(text(), '{keyword}')" for keyword in keywords])
    # 최종 XPath
    xpath = f"//*[{xpath_conditions}]"
    # 2.Selenium을 활용해 요소 찾기
    dynamic_elements = driver.find_elements(By.XPATH, xpath)
    driver.execute_script("window.scrollTo(0, 1000)")
    
    # 각 요소가 어떤 키워드를 포함하는지 확인하기 위한 리스트
    matched_keywords = []
    
    #3-1. div 레이아웃 가져오기
    dynamic_blocks, css_blocks = find_div(dynamic_elements)
    #3-2. 어떤 키워드와 일치하는지 찾음
    for html_elements in dynamic_blocks:
        element_keywords = [keyword for keyword in keywords if keyword in html_elements]
        matched_keywords.append(", ".join(element_keywords))
    
    return dynamic_blocks, css_blocks, matched_keywords, dynamic_elements

In [3]:
# 가장 작은 div 레이아웃 가져오기
def find_div(elements):
    html = []
    css = []
    for element in elements:
        try:
            # 현재 요소를 포함하는 가장 가까운 div 찾기
            closest_div = element.find_element(By.XPATH, './ancestor::div[1]')
            html_code = closest_div.get_attribute('outerHTML')
            
            # CSS 스타일 가져오기
            css_style = driver.execute_script("""
                const styles = window.getComputedStyle(arguments[0]);
                const styleObject = {};
                for (let i = 0; i < styles.length; i++) {
                    const propName = styles[i];
                    styleObject[propName] = styles.getPropertyValue(propName);
                }
                return styleObject;
            """, closest_div)
            
            # 스타일 출력
            style_output = "CSS Style:\n"
            for style_name, style_value in css_style.items():  # 딕셔너리의 각 항목
                style_output += f"{style_name}: {style_value}\n"
            
            # 결과 출력
            print("HTML Code:")
            print(html_code)
            print(style_output)
            
            #결과값 저장
            html.append(html_code)
            css.append(style_output)
            
        except NoSuchElementException:
            print("가장 가까운 div 요소를 찾을 수 없습니다.")
            
    return html, css

In [4]:
#가장 작은 div 레이아웃 가져오기 (static _ beautifulSoup)
def find_static_div(elements):
    static_html = []
    for element in elements:
        try:
            closet_div = element.find_parent('div')
            html_code = str(closet_div) #DOM 형식을 문자열로 변환
            print(f"HTML Code: {html_code}") #결과 출력
            static_html.append(html_code) #결과 저장
            
        except NoSuchElementException:
            print("가장 가까운 div 요소를 찾을 수 없습니다.")
            
    return static_html

In [42]:
# 동적 페이지와 동일한 부분 가져오기(텍스트 기반 탐색)
def search_false(dynamic_elements,url):
    
    print("Static URL open: "+ url)
    static_url = urlopen(url) #정적페이지
    static_html = BeautifulSoup(static_url, "html.parser")  # BeautifulSoup으로 파싱
    time.sleep(random.randrange(10,20)) #로딩되는데 걸리는 시간(10초 이상)
    
    dynamic = []
    static = []
    result = []
    
    for dynamic_element in dynamic_elements:
        try:
            # WebElement에서 텍스트 추출
            dynamic_text = dynamic_element.text.strip()  # 텍스트를 추출하고 공백 제거
            print(f"{dynamic_element}에서 추출된 텍스트는 다음과 같습니다. --> {dynamic_text}")

            #추출한 텍스트가 공백인 경우 제외 -> div를 찾을 수 없는 경우 해당
            if (dynamic_text == ''):
                print("공백은 제외합니다!")
                continue

            false_result = "false"
            static_parent = None
        
            #정적 페이지의 텍스트 요소 전체와 동적 페이지 텍스트 비교해 일치하는 것 찾기
            static_elements = static_html.find_all(string=lambda text: dynamic_text in text if text else False)
            static_div = find_static_div(static_elements)
         
            if len(static_div)>0 :
                static.append(static_div) #2차원 배열로 저장
            
            else:
                static.append([]) #2차원 배열로 저장 -> dynamic에 일치하는 것이 없어도 순서를 위해 배열을 채움
            
        except StaleElementReferenceException:
            print("Stale element reference encountered. Retrying...")
            continue
        
    return static

In [43]:
#1차원 배열과 2차원 배열 알맞게 매칭 후 저장
def data_frame(dynamic_html, static_html, css_code,keyword_matched):
    data = []
    for dynamic, statics, css, matched in zip(dynamic_html, static_html, css_code,keyword_matched): #두 배열을 병렬로 묶어 진행
        if statics:
            for static in statics: #statics은 1차원 배열
                data.append({'Dynamic_HTML': dynamic, 'Static_HTML':static, 'CSS': css, 'Matched Keywords': matched})
        else: #일치하는 문구가 없는 경우 공백 저장
            data.append({'Dynamic_HTML': dynamic, 'Static_HTML':None, 'CSS': css, 'Matched Keywords': matched})
            
    return data

# Main

In [54]:
# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_argument("Mozilla/5.0")  # 원하는 User-Agent 문자열로 변경

In [55]:
data = pd.read_csv('./data/false_test.csv')
driver = webdriver.Chrome()

# 결과 파일 존재 여부 확인
output_file = "./result/False_Check.csv"
file_exists = os.path.isfile(output_file)

In [56]:
#keyword 정의
user_alert = ["보고있", "보는중", " 보고 있", "보는 중"]
stock_alert = ["남음", "남았", "개 남음"]

keywords_dict = [user_alert, stock_alert]

In [57]:
for url in data['URL']:
    for keyword in keywords_dict:
        dynamic, css, matched, dynamic_elements = search_keyword(keyword,url, driver)
        static = search_false(dynamic_elements, url)

    # 데이터 저장
    data = data_frame(dynamic, static, css, matched)
    df = pd.DataFrame(data)
    
    #처음 저장 시에만 헤더를 추가, 이후에는 헤더 제외
    if not file_exists:
        df.to_csv(output_file, index=False, mode='w', header=True, encoding='utf-8')  # 첫 번째 URL
        file_exists = True  # 첫 번째 이후로는 헤더를 제외
    else:
        df.to_csv(output_file, index=False, mode='a', header=False, encoding='utf-8')  # 이후 URL

Dynamic URL open : https://www.11st.co.kr/products/7615498870
Static URL open: https://www.11st.co.kr/products/7615498870
Dynamic URL open : https://www.11st.co.kr/products/7615498870
Static URL open: https://www.11st.co.kr/products/7615498870
Dynamic URL open : https://www.agoda.com/ko-kr/search?city=16527&locale=ko-kr&ckuid=cbe5b914-1610-41bd-ab4d-111db75912cd&prid=0&currency=KRW&correlationId=5e3ddd71-186f-4493-85f2-2bc2a95ae2ad&analyticsSessionId=4589510054972858855&pageTypeId=1&realLanguageId=9&languageId=9&origin=KR&stateCode=11&cid=1844104&userId=cbe5b914-1610-41bd-ab4d-111db75912cd&whitelabelid=1&loginLvl=0&storefrontId=3&currencyId=26&currencyCode=KRW&htmlLanguage=ko-kr&cultureInfoName=ko-kr&machineName=hk-pc-2i-acm-web-user-6599bbcc8d-h9k4g&trafficGroupId=1&trafficSubGroupId=84&aid=130589&useFullPageLogin=true&cttp=4&isRealUser=true&mode=production&browserFamily=Chrome&cdnDomain=agoda.net&checkIn=2024-10-21&checkOut=2024-10-25&rooms=1&adults=2&children=0&priceCur=KRW&los=4&te

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=129.0.6668.100)
Stacktrace:
	GetHandleVerifier [0x00007FF75B92B095+29557]
	(No symbol) [0x00007FF75B89FA50]
	(No symbol) [0x00007FF75B75B56A]
	(No symbol) [0x00007FF75B72FCC5]
	(No symbol) [0x00007FF75B7DEE07]
	(No symbol) [0x00007FF75B7F7E21]
	(No symbol) [0x00007FF75B7D6F33]
	(No symbol) [0x00007FF75B7A116F]
	(No symbol) [0x00007FF75B7A22D1]
	GetHandleVerifier [0x00007FF75BC5C96D+3378253]
	GetHandleVerifier [0x00007FF75BCA8497+3688311]
	GetHandleVerifier [0x00007FF75BC9D1CB+3642539]
	GetHandleVerifier [0x00007FF75B9EA6B6+813462]
	(No symbol) [0x00007FF75B8AAB5F]
	(No symbol) [0x00007FF75B8A6B74]
	(No symbol) [0x00007FF75B8A6D10]
	(No symbol) [0x00007FF75B895C1F]
	BaseThreadInitThunk [0x00007FF8766A7374+20]
	RtlUserThreadStart [0x00007FF87681CC91+33]


In [58]:
driver.quit()